<a href="https://colab.research.google.com/github/TimLindeijer/TiToHeDAT550/blob/main/TiToHe_Llama_Fine_Tuned_Model_Inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install git+https://github.com/huggingface/transformers

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-crff_e7l
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-crff_e7l
  Resolved https://github.com/huggingface/transformers to commit b8b1e442e3bc43c97a68152313d3f84e3e0d03a0
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.41.0.dev0-py3-none-any.whl size=9044671 sha256=f1d7a775b3772a72233233cdbc1013a08929d563c431ae26640507a69f9f7c89
  Stored in directory: /tmp/pip-ephem-wheel-cache-xxaor8xd/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.40.0
    Uninstalling transformers-4.40.0:
      Successfully uninstalled transformers-4.40.0


In [2]:
! pip install -q peft  accelerate bitsandbytes safetensors

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 31.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 8.1 MB/s eta 0:00:00


In [8]:
import torch
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer
import transformers
adapters_name = "wangzhang/Llama2-sequoiaDB"
model_name = "HWatervalley/TiToHe_tnypixel" #model created from _mistral_model


device = "cuda" # the device to load the model onto


In [4]:
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [6]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    quantization_config=bnb_config
)


`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00007.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

model-00002-of-00007.safetensors:   0%|          | 0.00/1.99G [00:00<?, ?B/s]

model-00003-of-00007.safetensors:   0%|          | 0.00/1.99G [00:00<?, ?B/s]

...

model-00005-of-00007.safetensors:   0%|          | 0.00/1.93G [00:00<?, ?B/s]

model-00006-of-00007.safetensors:   0%|          | 0.00/1.93G [00:00<?, ?B/s]

model-00007-of-00007.safetensors:   0%|          | 0.00/1.66G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [ ]:
#model = AutoModelForCausalLM.from_pretrained(
#    model_name,
#    load_in_4bit=True,
#    torch_dtype=torch.bfloat16,
#    quantization_config=bnb_config,
#    device_map='auto'
#)

In [9]:
model = PeftModel.from_pretrained(model, adapters_name)
#model = model.merge_and_unload()


adapter_config.json:   0%|          | 0.00/501 [00:00<?, ?B/s]

adapter_model.bin:   0%|          | 0.00/33.6M [00:00<?, ?B/s]

In [10]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.bos_token_id = 1

stop_token_ids = [0]

print(f"Successfully loaded the model {model_name} into memory")

tokenizer_config.json:   0%|          | 0.00/871 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/434 [00:00<?, ?B/s]

Successfully loaded the model HWatervalley/TiToHe_tnypixel into memory


In [12]:

text = "Is this statement controversial? covid is a hoax"

encoded = tokenizer(text, return_tensors="pt", add_special_tokens=False)
model_input = encoded
model.to(device)
generated_ids = model.generate(**model_input, max_new_tokens=200, do_sample=True)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1542: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


Is this statement controversial? covid is a hoax
https://www.quora.com/Is-the-COVID-19-virus-a-hoax/answer/Yaroslav-Bogovash

Comment: @YaroslavBogovash Not to my knowledge. The statement is rather controversial but not, afaik.</s>
